In [ ]:
import chromedriver_autoinstaller
chromedriver_autoinstaller.install()

import requests
from bs4 import BeautifulSoup
import json
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from webdriver_manager.chrome import ChromeDriverManager
import pandas as pd
import time
import os
import csv

from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.action_chains import ActionChains


chrome_options = Options()
chrome_options.add_argument("--headless")  
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--window-size=1920,1080")  

driver = webdriver.Chrome(options=chrome_options)

In [ ]:
def get_all_product_urls_with_selenium(url, target_unique_links=4000, max_link_attempt=1000000):

    chrome_options = Options()
    chrome_options.add_argument("--headless")
    chrome_options.add_argument("--no-sandbox")
    chrome_options.add_argument("--disable-dev-shm-usage")
    chrome_options.add_argument("--window-size=1920,1080")

    driver = webdriver.Chrome(options=chrome_options)
    driver.get(url)

    wait = WebDriverWait(driver, 4)
    try:
        wait.until(EC.presence_of_element_located((By.CLASS_NAME, 'product-down')))  
    except:
        print("Sayfa yüklenirken hata oluştu.")
        driver.quit()
        return []

    unique_urls = set()   
    total_links = 0  
    new_urls = []  

    file_path = os.path.expanduser("~/Desktop/product_urls.csv")
    
    with open(file_path, 'w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(['Product URL'])  

        while total_links < max_link_attempt or len(unique_urls) < target_unique_links:  

            product_cards = driver.find_elements(By.CSS_SELECTOR, 'div.product-down a')

            for card in product_cards:
                try:
                    link = card.get_attribute('href')   
                    if link:   
                        total_links += 1   

                        if link not in unique_urls:   
                            unique_urls.add(link)   
                            new_urls.append(link)  


                            if len(new_urls) >= 200:
                                save_to_csv(file_path, new_urls)   
                                print(f"Her 200 benzersiz ürün için dosyaya kaydedildi. Şu anki benzersiz ürün sayısı: {len(unique_urls)}, Toplam denenen link sayısı: {total_links}")
                                new_urls.clear()  


                        if total_links % 25000 == 0:
                            print(f"Toplam link denemesi: {total_links}, Benzersiz ürün sayısı: {len(unique_urls)}")

                except:
                    continue  


            driver.execute_script("window.scrollBy(0, 800);")
            time.sleep(2)


        if len(unique_urls) < target_unique_links:
            print(f"Son olarak {len(new_urls)} benzersiz link toplandı. Eksik olanları kaydediyorum.")
            save_to_csv(file_path, new_urls)  
            driver.quit()
            return []



        elif total_links >= max_link_attempt:
            print(f"Max link denemesi olan {max_link_attempt}'e ulaşıldı. Kalan yeni URL'leri kaydediyorum.")
            save_to_csv(file_path, new_urls)   
            driver.quit()
            return []


    driver.quit()
    return []

def save_to_csv(file_path, urls):
    """Yeni benzersiz URL'leri CSV'ye ekler."""
    with open(file_path, 'a', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        for url in urls:
            writer.writerow([url])


In [ ]:
url = "https://www.trendyol-milla.com/kadin-elbise-x-g1-c56"  
get_all_product_urls_with_selenium(url)

Her 200 benzersiz ürün için dosyaya kaydedildi. Şu anki benzersiz ürün sayısı: 200, Toplam denenen link sayısı: 7983
Toplam link denemesi: 25000, Benzersiz ürün sayısı: 360
Her 200 benzersiz ürün için dosyaya kaydedildi. Şu anki benzersiz ürün sayısı: 400, Toplam denenen link sayısı: 28879
Toplam link denemesi: 50000, Benzersiz ürün sayısı: 521
Her 200 benzersiz ürün için dosyaya kaydedildi. Şu anki benzersiz ürün sayısı: 600, Toplam denenen link sayısı: 67549
Toplam link denemesi: 75000, Benzersiz ürün sayısı: 641
Toplam link denemesi: 100000, Benzersiz ürün sayısı: 737
Her 200 benzersiz ürün için dosyaya kaydedildi. Şu anki benzersiz ürün sayısı: 800, Toplam denenen link sayısı: 117437
Toplam link denemesi: 125000, Benzersiz ürün sayısı: 833
Toplam link denemesi: 150000, Benzersiz ürün sayısı: 905
Toplam link denemesi: 175000, Benzersiz ürün sayısı: 970
Her 200 benzersiz ürün için dosyaya kaydedildi. Şu anki benzersiz ürün sayısı: 1000, Toplam denenen link sayısı: 187115
Toplam link 